# Houses task solver

In [85]:
import numpy as np
import copy
import re
from requests import get

In [97]:
def load_task(file_location):
    contents = get(file_location).text
    contents = re.findall('[0-9]+', contents)
    row = []
    plane = []
    col = [int(char) for char in contents[0]]
    for i in range(1, len(contents)):
        row.append(int(contents[i][0]))
        plane.append([int(contents[i][j]) for j in range(1, len(contents[i]))])
    plane = np.array(plane)
    
    return row, col, plane

def print_task(row, col, plane):
    text = '  ' + ' '.join(str(x) for x in col) + '\n  ' + '_' * (2 * len(col) - 1) + '\n'
    for i in range(len(row)):
        text += str(row[i]) + '|' + ' '.join(str(x) for x  in plane[i]) + '\n'
    print(text)

In [99]:
row, col, game_plane = load_task('https://raw.githubusercontent.com/gekas145/sudoku/main/houses/task2.txt')
print_task(row, col, game_plane)

  3 2 0 5 0 5 0 2 2 3 0 2 2 2
  ___________________________
5|0 0 0 0 1 0 1 1 0 0 1 0 0 1
1|1 0 0 0 0 0 0 0 0 0 0 0 0 0
4|0 0 0 0 1 0 0 0 1 0 0 1 0 0
1|0 1 0 0 1 0 0 0 0 0 0 0 0 0
4|0 0 0 0 1 0 0 0 0 0 0 0 0 0
2|1 0 1 0 0 1 0 0 1 1 0 1 1 0
4|1 0 0 0 0 0 0 1 0 0 0 0 1 0
2|0 0 0 0 1 0 0 0 0 0 0 0 0 0
1|0 0 0 0 0 0 0 0 1 0 0 0 1 0
4|0 1 1 0 0 0 1 0 0 0 0 0 0 0

